In [17]:
import pandas as pd
import re
import numpy as np
import os

In [3]:
def importing_all_matches(directory, championship_dict, teams_dict, all_matches = True):
    for filename in os.listdir(directory): #For para mundiais
        if filename.startswith("worlds"):
            insert_dataframe(directory + "/", filename, "worlds",championship_dict)
    for filename in os.listdir(directory): #For para fase de grupos 
        if filename.endswith("fg.csv"):
            insert_dataframe(directory + "/", filename, "fg",championship_dict)
    for filename in os.listdir(directory): #For para play offs (preciso rodar dois fors aqui para que eu nao tente adicionar play offs em um df de um campeonato que ainda nao teve fase de grupos)
         if filename.endswith("po.csv") and all_matches:
            insert_dataframe(directory + "/", filename, "po",championship_dict)
    create_teams_dict(championship_dict,teams_dict)
            
            
def insert_dataframe(directory, filename, stage, championship_dict):
    a = pd.read_csv(directory + filename)
    Winner = []
    Loser = []
    Week = []
    #Side = []
    for i in a['Team 1;"Team 2";"Score";"Etapa"']:
        k= re.split(r'[;"][;"\s*]', i)
        try:
            t1, t2, placar1, placar2 = (k[0].upper(), k[1].upper(), int(k[2][1]), int(k[2][-1]))
        except:
            break

        if stage == "fg":
            try:
                week = int(k[3][-3:-1])
            except:
                week = Week[-1]   #tiebreakers 
        elif stage == "po":
            week = -1        
        elif stage == "worlds":
            week = int(filename[-8:-4])

        for i in range(0,placar1):
            Winner.append(t1)
            Loser.append(t2)
            Week.append(week)
            #Side.append("Blue")
        for i in range(0,placar2):
            Winner.append(t2)
            Loser.append(t1)
            Week.append(week)
            #Side.append("Red")
    data = {"Winner":Winner, "Loser":Loser, "Week":Week} if stage != "worlds" else {"Winner":Winner, "Loser":Loser, "Year":Week}
    game_table = pd.DataFrame(data)
    cols = ["Winner", "Loser", "Week"] if stage != "worlds" else ["Winner", "Loser", "Year"]
    df_name = int(filename[-8:-4]) if stage == "worlds" else filename[0:3]
    if stage == "fg" or stage == "worlds":
        championship_dict[df_name] = game_table[cols]
    elif stage == "po":
        week = -1
        championship_dict[df_name] = championship_dict[df_name].append(game_table[cols], ignore_index = True)
        
def create_teams_dict(championship_dic, teams_dict):
    for cs in championship_dic.keys():
        a = list(championship_dic[cs].Winner)
        a.extend(list(championship_dic[cs].Loser))
        teams_dict[cs] = set(a)
        
def update_teams(teams_dict, initials_dict):
    fake_Times = Times
    for reg in teams_dict:
        for team in teams_dict[reg]:
            if team not in set(fake_Times.Sigla):
                newteam = pd.DataFrame([[team, initials_dict[reg]]], columns=["Sigla", "Regiao"])
                fake_Times = fake_Times.append(newteam, ignore_index=True)
            elif initials_dict[reg] != str(fake_Times[fake_Times.Sigla==team].Regiao):             
                fake_Times.iloc[fake_Times[fake_Times.Sigla==team].index, fake_Times.columns.get_loc('Regiao')] = initials_dict[reg]
    return fake_Times
#             if str(Times[Times.Sigla == team].Regiao) != initials_dict[reg]:
#                 number = Times[Times.Sigla == team].index[0]
#                 print(team, number, Times.iloc[number].Regiao, initials_dict[reg])